In [1]:
from xdsl.dialects.builtin import *
from xdsl.printer import Printer
from devito.ir.ietxdsl import *
from xdsl.dialects.builtin import ModuleOp

In [2]:
from devito import Grid, TimeFunction, Eq, Operator

In [3]:
from devito.ir.iet.visitors import Visitor

In [4]:
import devito.ir.ietxdsl as ietxdsl

In [5]:
def test_example():
    ctx = MLContext()
    Builtin(ctx)
    iet = IET(ctx)

    mod = ModuleOp.from_region_or_ops([
        Callable.get("kernel", ["u"], Block.from_callable([iet.i32], lambda u: [
            Iteration.get(["affine", "sequential"], ("time_m", "time_M", "1"),
                          Block.from_callable([iet.i32, iet.i32, iet.i32], lambda time, t0, t1: [
                Iteration.get(["affine", "parallel", "skewable"], ("x_m", "x_M", "1"),
                              Block.from_callable([iet.i32], lambda x: [
                    Iteration.get(["affine", "parallel", "skewable", "vector-dim"], ("y_m", "y_M", "1"),
                                  Block.from_callable([iet.i32], lambda y: [
                        cst1    := Constant.get(1),
                        x1      := Addi.get(x, cst1),
                        y1      := Addi.get(y, cst1),
                        ut0     := Idx.get(u, t0),
                        ut0x1   := Idx.get(ut0, x1),
                        ut0x1y1 := Idx.get(ut0x1, y1),
                        rhs     := Addi.get(ut0x1y1, cst1),
                        ut1     := Idx.get(u, t1),
                        ut1x1   := Idx.get(ut1, x1),
                        lhs     := Idx.get(ut1x1, y1),
                        Assign.build([lhs, rhs])
                    ]))
                ]))
            ]))
        ]))
    ])
    
    printer = Printer()
    printer.print_op(mod)

test_example()

module() {
  iet.callable() ["callable_name" = "kernel", "parameters" = ["u"]] {
  ^0(%0 : !i32):
    iet.iteration() ["limits" = ["time_m", "time_M", "1"], "properties" = ["affine", "sequential"]] {
    ^1(%1 : !i32, %2 : !i32, %3 : !i32):
      iet.iteration() ["limits" = ["x_m", "x_M", "1"], "properties" = ["affine", "parallel", "skewable"]] {
      ^2(%4 : !i32):
        iet.iteration() ["limits" = ["y_m", "y_M", "1"], "properties" = ["affine", "parallel", "skewable", "vector-dim"]] {
        ^3(%5 : !i32):
          %6 : !i32 = iet.constant() ["value" = 1 : !i32]
          %7 : !i32 = iet.addi(%4 : !i32, %6 : !i32)
          %8 : !i32 = iet.addi(%5 : !i32, %6 : !i32)
          %9 : !i32 = iet.idx(%0 : !i32, %2 : !i32)
          %10 : !i32 = iet.idx(%9 : !i32, %7 : !i32)
          %11 : !i32 = iet.idx(%10 : !i32, %8 : !i32)
          %12 : !i32 = iet.addi(%11 : !i32, %6 : !i32)
          %13 : !i32 = iet.idx(%0 : !i32, %3 : !i32)
          %14 : !i32 = iet.idx(%13 : !i32, %7 : !i32

In [6]:
grid = Grid(shape = (3, 3))
u = TimeFunction(name='u', grid=grid)
eq = Eq(u.forward, u + 1)
op = Operator([eq])
print(op.view)

<Callable Kernel>
  <CallableBody <allocs=0, casts=1, maps=0> <unmaps=0, frees=0>>
    <List (4, 0, 0)>
      <C.Comment /* Flush denormal numbers to zero in hardware */>
      <C.Statement _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);>
      <C.Statement _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);>
      <C.Line >


    <List (0, 1, 0)>

      <[affine,sequential] Iteration time::time::(time_m, time_M, 1)>
        <Section (section0)>

          <TimedList (1, 1, 1)>
            <[affine,parallel,parallel=,skewable] Iteration x::x::(x_m, x_M, 1)>
              <[affine,parallel,parallel=,skewable,vector-dim] Iteration y::y::(y_m, y_M, 1)>
                <ExpressionBundle (1)>

                  <Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 1>





In [7]:
print(op)

#define _POSIX_C_SOURCE 200809L
#define START_TIMER(S) struct timeval start_ ## S , end_ ## S ; gettimeofday(&start_ ## S , NULL);
#define STOP_TIMER(S,T) gettimeofday(&end_ ## S, NULL); T->S += (double)(end_ ## S .tv_sec-start_ ## S.tv_sec)+(double)(end_ ## S .tv_usec-start_ ## S .tv_usec)/1000000;

#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict u_vec, const int time_M, const int time_m, const int x_M, const int x_m, const int y_M, const int y_m, struct profiler * timers)
{
  float (*restrict u)[u_vec->size[1]][u_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]]) u_vec->data;

  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON)

In [8]:
expr = op.body.body[1].body[0].children[0][0].body[0].body[0].children[0][0].children[0][0].children[0][0].expr
expr

Eq(u[t1, x + 1, y + 1], u[t0, x + 1, y + 1] + 1)

In [9]:
# Transform n-dimensional index expressions to n 1-dimensional index expressions
from sympy import Indexed, IndexedBase, symbols, Integer, Symbol, Add, Mul, Eq
import sympy
from devito.types.basic import IndexedData

def add_to_block(expr, arg_by_expr, result):
    if expr in arg_by_expr:
        return
    
    if isinstance(expr, IndexedData):
        # Only index first bit of IndexedData
        add_to_block(expr.args[0], arg_by_expr, result)
        arg_by_expr[expr] = arg_by_expr[expr.args[0]]
        return
    
    if isinstance(expr, Symbol):
        # All symbols must be passed in at the start
        my_expr = Symbol(expr.name)
        assert my_expr in arg_by_expr, f'Symbol with name {expr.name} not found in {arg_by_expr}'
        arg_by_expr[expr] = arg_by_expr[my_expr]
        return
    
    if isinstance(expr, Integer):
        constant = int(expr.evalf())
        arg = Constant.get(constant)
        arg_by_expr[expr] = arg
        result.append(arg)
        return

    for child_expr in expr.args:
        add_to_block(child_expr, arg_by_expr, result)
    
    if isinstance(expr, Add):
        lhs = arg_by_expr[expr.args[0]]
        rhs = arg_by_expr[expr.args[1]]
        sum = Addi.get(lhs, rhs)
        arg_by_expr[expr] = sum
        result.append(sum)
        return
    
    if isinstance(expr, Indexed):
        add_to_block(expr.args[0], arg_by_expr, result)
        prev = arg_by_expr[expr.args[0]]
        for child_expr in expr.args[1:]:
            add_to_block(child_expr, arg_by_expr, result)
            child_arg = arg_by_expr[child_expr]
            idx = Idx.get(prev, child_arg)
            result.append(idx)
            prev = idx
        arg_by_expr[expr] = prev
        return
    
    if isinstance(expr, Eq):
        add_to_block(expr.args[0], arg_by_expr, result)
        lhs = arg_by_expr[expr.args[0]]
        add_to_block(expr.args[1], arg_by_expr, result)
        rhs = arg_by_expr[expr.args[1]]
        assign = Assign.build([lhs, rhs])
        arg_by_expr[expr] = assign
        result.append(assign)
        return

    assert False, f'unsupported expr {expr} of type {expr.func}'


In [10]:
import devito.ir.iet.nodes as nodes

In [11]:
ctx = MLContext()
Builtin(ctx)
iet = IET(ctx)

IET_TYPES = {'int': iet.i32}

def myVisit(node, block=None, ctx={}):
    assert isinstance(node, nodes.Node), f'Argument must be subclass of Node, found: {node}'
    
    if hasattr(node, 'is_Callable') and node.is_Callable:
        name = node.name
        parameters = node.parameters
        body = myVisit(node.body)
        return
    
    if isinstance(node, nodes.CallableBody):
#         print(f'CallableBody: f{node.view}')
        body = [myVisit(x) for x in node.body]
        return 
    
    if isinstance(node, nodes.Expression):
        expr = node.expr
        r = []
        add_to_block(expr, {Symbol(s): a for s, a in ctx.items()}, r)
        block.add_ops(r)
        return
    
    if isinstance(node, nodes.ExpressionBundle):
        assert len(node.children) == 1
        assert len(node.children[0]) == 1
        myVisit(node.children[0][0], block, ctx)
        return
    
    if isinstance(node, nodes.Iteration):
        assert len(node.children) == 1
        assert len(node.children[0]) == 1
        index = node.index
        b = Block.from_arg_types([iet.i32])
        ctx = {**ctx, index: b.args[0]}
        myVisit(node.children[0][0], b, ctx)
        iteration = Iteration.get(node.properties, node.limits, b)
        block.add_ops([iteration])
        return
    
    raise TypeError(f'Unsupported type of node: {type(node)}, {vars(node)}')

ints = ['t0', 't1', 'x', 'u']
b = Block.from_arg_types([iet.i32] * len(ints))
d = {name: register for name, register in zip(ints, b.args)}
    
node = op.body.body[1].body[0].children[0][0].body[0].body[0]
print(vars(node))
# node.index
myVisit(node, block=b, ctx=d)

Printer()._print_named_block(b)

{'_args': {'nodes': (<WithProperties[affine,parallel,parallel=,skewable,vector-dim]::Iteration y; (y_m, y_M, 1)>,), 'dimension': x, 'limits': (x_m, x_M, 1), 'direction': ++, 'properties': {parallel=, affine, parallel, skewable}, 'uindices': (), 'pragmas': None}, 'nodes': (<WithProperties[affine,parallel,parallel=,skewable,vector-dim]::Iteration y; (y_m, y_M, 1)>,), 'dim': x, 'index': 'x', 'direction': ++, 'limits': (x_m, x_M, 1), 'properties': (affine, parallel, parallel=, skewable), 'pragmas': (), 'uindices': ()}
^0(%0 : !i32, %1 : !i32, %2 : !i32, %3 : !i32):
  iet.iteration() ["limits" = ["x_m", "x_M", "1"], "properties" = ["affine", "parallel", "parallel=", "skewable"]] {
  ^1(%4 : !i32):
    iet.iteration() ["limits" = ["y_m", "y_M", "1"], "properties" = ["affine", "parallel", "parallel=", "skewable", "vector-dim"]] {
    ^2(%5 : !i32):
      %6 : !i32 = iet.constant() ["value" = 1 : !i32]
      %7 : !i32 = iet.addi(%6 : !i32, %4 : !i32)
      %8 : !i32 = iet.addi(%6 : !i32, %5 : 

In [12]:
vars(op)

{'_args': {'name': 'Kernel',
  'body': <CallableBody <allocs=0, casts=1, maps=0> <unmaps=0, frees=0>>,
  'retval': 'int',
  'parameters': [u(t, x, y), time_M, time_m, x_M, x_m, y_M, y_m, timers],
  'prefix': ()},
 'name': 'Kernel',
 'body': <CallableBody <allocs=0, casts=1, maps=0> <unmaps=0, frees=0>>,
 'retval': 'int',
 'prefix': (),
 'parameters': (u(t, x, y), time_M, time_m, x_M, x_m, y_M, y_m, timers),
 '_headers': [('_POSIX_C_SOURCE', '200809L'),
  ('START_TIMER(S)',
   'struct timeval start_ ## S , end_ ## S ; gettimeofday(&start_ ## S , NULL);'),
  ('STOP_TIMER(S,T)',
   'gettimeofday(&end_ ## S, NULL); T->S += (double)(end_ ## S .tv_sec-start_ ## S.tv_sec)+(double)(end_ ## S .tv_usec-start_ ## S .tv_usec)/1000000;')],
 '_globals': [],
 '_includes': ['stdlib.h',
  'math.h',
  'sys/time.h',
  'xmmintrin.h',
  'pmmintrin.h'],
 '_compiler': JITCompiler[GNUCompiler],
 '_lib': None,
 '_cfunction': None,
 '_func_table': OrderedDict(),
 '_state': {'optimizations': 'advanced'},
 '_inpu